In [43]:
import pandas as pd
import streamlit as st
import plotly.graph_objects as go
import urllib, json

In [80]:
funds = pd.read_csv('../data/processed/funds.csv', sep = ';', encoding = 'latin1', decimal=',')
dim_funds = pd.read_csv('../data/processed/dim_funds.csv', sep = ';', encoding = 'latin1', decimal=',')

In [119]:
fund = 'DAHLIA GLOBAL ALLOCATION MASTER FUNDO DE INVESTIMENTO MULTIMERCADO'
df = funds[funds['NM_FUNDO_COTA'] == fund][['CNPJ_FUNDO', 'DENOM_SOCIAL', 'VL_MERC_POS_FINAL', 'CNPJ_FUNDO_COTA', 'NM_FUNDO_COTA']].reset_index(drop = True)

In [120]:
funds_searched = [fund]


funds_search = list(df['DENOM_SOCIAL'].unique())


while funds_search:

    df_search = funds[funds['NM_FUNDO_COTA'].isin(funds_search)][['CNPJ_FUNDO', 'DENOM_SOCIAL', 'VL_MERC_POS_FINAL', 'CNPJ_FUNDO_COTA', 'NM_FUNDO_COTA']].reset_index(drop = True)

    funds_searched = funds_searched + funds_search

    funds_search = list(df_search['DENOM_SOCIAL'].unique())


    funds_search = set(funds_search) - set(funds_searched)

    funds_search = list(funds_search)



    df = pd.concat([df_search, df])





In [121]:
dim_sankey = pd.concat([df[['CNPJ_FUNDO', 'DENOM_SOCIAL']], df[['CNPJ_FUNDO_COTA', 'NM_FUNDO_COTA']].rename(columns={'CNPJ_FUNDO_COTA' : 'CNPJ_FUNDO', 'NM_FUNDO_COTA' : 'DENOM_SOCIAL'})]).drop_duplicates().reset_index(drop = True).reset_index()
dim_sankey

,index,CNPJ_FUNDO,DENOM_SOCIAL
0,0,31081028000124,SANTA MARIA FUNDO DE INVESTIMENTO MULTIMERCADO...
1,1,5069379000100,PORT MM MULTIGESTORES FI EM COTAS DE FUNDOS DE...
2,2,21888751000166,M&T FUNDO DE INVESTIMENTO EM AÇÕES
3,3,19366036000111,PAT FUTURO FUNDO DE INVESTIMENTO MULTIMERCADO ...
4,4,22032175000113,TOMMA FUNDO DE INVESTIMENTO EM COTAS DE FI MUL...
5,5,18630109000178,FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRI...
6,6,26725153000190,GSFC FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDIT...
7,7,28386172000164,LE MONDE FI EM COTAS DE FUNDOS DE INVESTIMENTO...
8,8,29725797000176,SARARA FUNDO DE INVESTIMENTO EM COTAS DE FUNDO...
9,9,30330454000191,ASPEN HIGHLANDS FICFI MULTIMERCADO CRÉDITO PRI...


In [122]:
df = df.merge(dim_sankey, left_on='CNPJ_FUNDO', right_on='CNPJ_FUNDO').rename(columns={'index' : 'index_source'})
df = df.merge(dim_sankey, left_on='CNPJ_FUNDO_COTA', right_on='CNPJ_FUNDO').rename(columns={'index' : 'index_target'}).drop(columns=['CNPJ_FUNDO_y'], axis=1).rename(columns={'CNPJ_FUNDO_x' : 'CNPJ_FUNDO'})

In [138]:
source = df['index_source']
target = df['index_target']
value = df['VL_MERC_POS_FINAL']
label = dim_sankey['DENOM_SOCIAL']
link = dict(source = source, target = target, value = value, label = label)
data = go.Sankey(link = link)

In [139]:
fig = go.Figure(data)
fig.show()

In [140]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    #valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  label
      #color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  source,
      target =  target,
      value =  value,
      #label =  data['data'][0]['link']['label'],
      #color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="CDA", font_size=10)
fig.show()

In [127]:
url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()